In [107]:
'''Author: Steve Walesch
Copyright: Copyright (C) 2017 Steve Walesch
License: Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
Version: 1.0 RC1'''

from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets
import numpy as np
import math
from ipythonblocks import show_color

In [108]:
def name_this_colour(Hue, Saturation, Lightness):

    # hue name
    hues = {0: 'red',
     15: 'red-orange',
     30: 'orange',
     45: 'orange-yellow',
     60: 'yellow',
     75: 'yellow-chartreuse',
     90: 'chartreuse',
     105: 'chatreuse-green',
     120: 'green',
     135: 'green-jade',
     150: 'jade',
     165: 'jade-cyan',
     180: 'cyan',
     195: 'cyan-azure',
     210: 'azure',
     225: 'azure-blue',
     240: 'blue',
     255: 'blue-violet',
     270: 'violet',
     285: 'violet-magenta',
     300: 'magenta',
     315: 'magenta-rose',
     330: 'rose',
     345: 'rose-red',
     360: 'red'}

    Hue_for_dict = min(hues, key=lambda x:abs(x-Hue))
    Hue_name = hues[Hue_for_dict]

    Saturation_dict = {0.10: 'grayish', 0.30: 'desaturated', 0.50: 'muted', 0.70: 'saturated', 0.90: 'vivid'}
    Saturation_for_dict = min(Saturation_dict, key=lambda x:abs(x-Saturation))
    Saturation_name = Saturation_dict[Saturation_for_dict]

    Lightness_dict = {0.10: 'and very dark', 0.30: 'dark', 0.50: '', 0.70001: 'light', 0.90001: 'and very light'}
    Lightness_for_dict = min(Lightness_dict, key=lambda x:abs(x-Lightness))
    Lightness_name = Lightness_dict[Lightness_for_dict]

    Colour_name = ''
    if Saturation == 0:

        if Lightness != 0 and Lightness !=1:

            if Lightness <= 0.20:
                Colour_name = 'very dark gray'
            elif Lightness > 0.20 and Lightness <= 0.40:
                Colour_name = 'dark gray'
            elif Lightness > 0.60 and Lightness <= 0.80:
                Colour_name = 'light gray'
            elif Lightness > 0.80 and Lightness <= 1.00:
                Colour_name = 'very light gray'
            else:
                Colour_name = 'medium gray'
        elif Lightness == 0:
                Colour_name = 'black'
        elif Lightness == 1:
                Colour_name = 'white'
    else:
        Colour_name = "%s %s %s" %(Saturation_name, Lightness_name, Hue_name)
    return Colour_name

In [109]:
def from_RGB(Red, Green, Blue):

    rgb_255 = np.array([Red,Green,Blue])
    rgb_percent = rgb_255 / 255
    diff = np.max(rgb_percent)-np.min(rgb_percent)
    red_pc = rgb_percent[0]
    green_pc = rgb_percent[1]
    blue_pc = rgb_percent[2]

    # calculate chromaticness, whiteness and blackness
    Chromaticness = np.max(rgb_percent) - np.min(rgb_percent)
    Whiteness = np.max(rgb_percent) - Chromaticness
    Blackness = 1 - Chromaticness - Whiteness
    print("Chromaticness: %.2f%%, Whiteness: %.2f%%, Blackness: %.2f%%" % (Chromaticness*100, Whiteness*100, Blackness*100))
    
    # calculate Hue, Saturation and Lightness
    if np.max(rgb_percent) == np.min(rgb_percent):
        Hue = 0
    elif np.max(rgb_percent) == red_pc:
        Hue = (60 * ((green_pc-blue_pc)/diff) + 360) % 360
    elif np.max(rgb_percent) == green_pc:
        Hue = (60 * ((blue_pc-red_pc)/diff) + 120) % 360
    elif np.max(rgb_percent) == blue_pc:
        Hue = (60 * ((red_pc-green_pc)/diff) + 240) % 360
        
    Lightness = Whiteness + Chromaticness/2
    Saturation = Chromaticness
    
    print("Hue: %.2f, Saturation: %.2f%%, Lightness: %.2f%%" % (Hue, Saturation*100, Lightness*100))
    # Visualize the colour
    show_color(Red, Green, Blue)
    
    # Name the colour
    print("This colour is named: %s" % (name_this_colour(Hue,Saturation,Lightness)))
    return None

In [110]:
# this function gives the RGB values for 'pure colours'. it is a helper function that's used in the chsl2rgb function
def pure_hue(hue):
    hue = hue % 1.0
    if hue < 1/6:
        return hue*6.0
    if hue < 0.5:
        return 1
    if hue < 2/3:
        return (2/3-hue)*6.0
    return 0

In [111]:
# define widgets with constraints for the following interaction
saturation_widget = FloatSlider(min=0.0, max=1.0, step=0.01)
lightness_widget = FloatSlider(min=0.0, max=1.0, step=0.01)

def update_lightness_range(*args):
    lightness_widget.min = 0.5 * saturation_widget.value
    lightness_widget.max = 1 - 0.5 * saturation_widget.value
saturation_widget.observe(update_lightness_range, 'value')

In [112]:
def to_RGB(Hue,Saturation,Lightness):
    r,g,b = (pure_hue(Hue/360+1/3)*255, pure_hue(Hue/360)*255, pure_hue(Hue/360-1/3)*255)
    #print("RGB values for fully saturated colour:", int(r),int(g),int(b))
    
    Chromaticness = Saturation
    Whiteness = Lightness - Saturation/2
    Blackness = 1 - Whiteness - Chromaticness
    print("Chromaticness: %.2f%%, Whiteness: %.2f%%, Blackness: %.2f%%" % (Chromaticness*100, Whiteness*100, Blackness*100))
    
    Red = int(r*Saturation + Whiteness * 255)
    Green = int(g*Saturation + Whiteness * 255)
    Blue = int(b*Saturation + Whiteness * 255)
    print("RGB values: Red: %s, Green: %s, Blue: %s" % (Red, Green, Blue))
    show_color(Red, Green, Blue)
    print("This colour is named: %s" % (name_this_colour(Hue,Saturation,Lightness)))
    return None

In [113]:
interact(to_RGB, Hue=(0,360,1), Saturation=saturation_widget, Lightness=lightness_widget); 

Chromaticness: 0.00%, Whiteness: 0.00%, Blackness: 100.00%
RGB values: Red: 0, Green: 0, Blue: 0


This colour is named: black


In [114]:
interact(from_RGB, Red=(0,255,1), Green=(0,255,1), Blue=(0,255,1))

Chromaticness: 0.00%, Whiteness: 49.80%, Blackness: 50.20%
Hue: 0.00, Saturation: 0.00%, Lightness: 49.80%


This colour is named: medium gray


<function __main__.from_RGB>